<a href="https://colab.research.google.com/github/DucLeTrong/genetic_algorithm/blob/master/GA_WRSNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
random.seed( 30 )
import math
paras = {}
f = open('2.txt','r')
list_node = []
for x in f:
    list_node.append([float(i) for i in x.split(" ")])
paras["list_node"] = list_node
def calcualte_distances(list_node):
    distances =[[None]*len(list_node) for i in range(len(list_node))] 
    for i in range(len(list_node)):
        for j in range(len(list_node)):
            distances[i][j] = math.sqrt((list_node[i][0]-list_node[j][0])**2 + (list_node[i][1]-list_node[j][1])**2)

    # print(distances)
    return distances
paras['distances'] = calcualte_distances(list_node)
# distances[1][2]
paras['V'] = 5
paras['ALPHA'] = 0.5
paras['num_of_gens'] = len(paras['list_node'])
paras['size_popu'] = 100
def calcualte_fitness(chromosome, distances, list_node):
    k = []
    for i in range(0,len(chromosome)):
        # w = list_node[chromosome[i]][3]/list_node[chromosome[i]][2]
        w = list_node[chromosome[i]][3]/list_node[chromosome[i]][2]
        if i==0:
            T = math.sqrt(list_node[chromosome[i]][0]**2 + list_node[chromosome[i]][1]**2) /paras['V']
        else:
            T = T + distances[i-1][i]/paras['V']
        k.append(T/w)
    
    f1 = sum(k)
    f2 = 0
    for i in k:
        f2 += abs(i - f1/len(chromosome))
    fitness = paras['ALPHA'] *f1 + (1 - paras['ALPHA'])  * f2
    return fitness

In [0]:
class Individual:
    def __init__(self, init_value):
        if isinstance(init_value, int):
            number_of_gene = init_value
            self.chromosome = list(range(number_of_gene))
            self.chromosome = sorted(self.chromosome, key=lambda *args: random.random())
            self.fitness_score = calcualte_fitness(self.chromosome, paras['distances'], list_node)
        else:
            self.chromosome = init_value
            self.fitness_score = calcualte_fitness(self.chromosome, paras['distances'], list_node)

    def get_fitness(self):
        return self.fitness_score

    def get_size(self):
        return len(self.chromosome)

    def swap_mutate(self):
        point1 = int(random.random() * self.get_size())
        point2 = int(random.random() * self.get_size())
        while point2==point1:
            point2 = int(random.random() * self.get_size())
        self.chromosome[point1], self.chromosome[point2] = self.chromosome[point2], self.chromosome[point1]
    
    def cim_mutate(self):
        point = int(random.random() * self.get_size())
        print(point)
        gene = self.chromosome[point::-1]
        gene.extend(self.chromosome[-1:point:-1])
        self.chromosome = gene

    def mutate(self):
        if random.random() < 0.4:
            self.swap_mutate()
        else:
            self.cim_mutate()

In [0]:
class Population():
    def __init__(self, size_population, num_of_gen):
        # self.size = size
        self.individuals = []
        for i in range(size_population):
            individual = Individual(num_of_gen)
            self.individuals.append(individual)
        print(len(self.individuals))

    def get_size(self):
        return len(self.individuals)

    def get_individual(self, index):
      return self.individuals[index]

    def add_individual(self, individual):
        self.individuals.append(individual)


    def get_fittest(self):
        fittest = self.individuals[0]
        # print("len", self.get_size())
        for i in range(self.get_size()):
            # print("dasd",i,self.individuals[i].get_fitness())
            if fittest.get_fitness() > self.individuals[i].get_fitness():
                fittest = self.get_individual(i)
                
        # self.individuals = s
        return fittest

In [0]:
def pmx(parent1, parent2, start, stop):
    child = [None]*parent1.get_size()
    child[start:stop] = parent1.chromosome[start:stop]
    for ind,x in enumerate(parent2.chromosome[start:stop]):
        ind += start
        if x not in child:
            while child[ind] != None:
                ind = parent2.chromosome.index(parent1.chromosome[ind])
            child[ind] = x
    for ind,x in enumerate(child):
        if x == None:
            child[ind] = parent2.chromosome[ind]
    child_idi = Individual(child)
    return child_idi
    
def pmx_pair(a,b):
    half = a.get_size() // 2
    start = random.randint(0, a.get_size()-half)
    stop = start + half
    return pmx(a,b,start,stop) , pmx(b,a,start,stop)
def simple_crossover( parent1, parent2):
        num_of_gene = parent1.get_size()
        cutting_point = random.randint(0,num_of_gene-1)
        chromosome1 = parent1.chromosome[:cutting_point]
        chromosome2 = parent2.chromosome[:cutting_point]

        for i in range(num_of_gene):
            # print(i)
            if parent1.chromosome[i] not in chromosome2:
                chromosome2.append(parent1.chromosome[i])
            if parent2.chromosome[i] not in chromosome1:
                chromosome1.append(parent2.chromosome[i])
        child1 = Individual(chromosome1)
        child2 = Individual(chromosome2)
        return child1, child2

In [0]:
class GA:
    def crossover(self, ind1, ind2):
        if random.random() < 0.6:
            return pmx_pair(ind1,ind2)
        else:
            return simple_crossover(ind1, ind2)

    def crossover_population(self, population, num_of_pair):
        for i in range(num_of_pair):
            j1 = random.randint(0,population.get_size()-1)
            j2 = random.randint(0,population.get_size()-1)
            while j1==j2:
                j2 = random.randint(0,population.get_size()-1)


            child1, child2 = self.crossover(population.individuals[j1], population.individuals[j2])
            population.add_individual(child1)
            population.add_individual(child2)

    def mutate_population(self, population):
        for individual in population.individuals:
            if random.random() < 0.2:
                individual.swap_mutate()
    
    def natureSelection(self, population,size):
        population.individuals = sorted(population.individuals, key=Individual.get_fitness)[:size]


    def run(self, population, number_loop):
        for i in range(number_loop):
            self.crossover_population(population,100)
            self.mutate_population(population)
            self.natureSelection(population,250)
            print("loop", i, "best fitness",population.get_fittest().get_fitness())
        print("Best solotion",population.get_fittest().get_fitness())
        print("Best solotion",population.get_fittest().chromosome)


In [7]:
random.seed(30)
paras['number_loop'] = 3000
popualation = Population(paras['size_popu'] ,paras['num_of_gens'])
ga = GA()
ga.run(popualation, paras['number_loop'])

100
loop 0 best fitness 32.98625638563986
loop 1 best fitness 32.97089466755239
loop 2 best fitness 31.849272373920336
loop 3 best fitness 31.724083771914373
loop 4 best fitness 29.000945839577444
loop 5 best fitness 29.000945839577444
loop 6 best fitness 29.000945839577444
loop 7 best fitness 29.000945839577444
loop 8 best fitness 29.000945839577444
loop 9 best fitness 29.000945839577444
loop 10 best fitness 27.552587965751044
loop 11 best fitness 27.552587965751044
loop 12 best fitness 27.387040464725136
loop 13 best fitness 27.387040464725136
loop 14 best fitness 27.387040464725136
loop 15 best fitness 27.387040464725136
loop 16 best fitness 27.33128218517745
loop 17 best fitness 27.322470605090288
loop 18 best fitness 27.322470605090288
loop 19 best fitness 27.26094725929169
loop 20 best fitness 27.030280377251614
loop 21 best fitness 26.744138240130987
loop 22 best fitness 25.427043790911277
loop 23 best fitness 25.427043790911277
loop 24 best fitness 25.427043790911277
loop 25 be